현재 상태:

Tumor 디렉토리:

    - SRR30510137: ✅ 완료

    - SRR30510151: ✅ 완료

    - SRR30510181: ✅ 완료

Normal 디렉토리:

    - SRR28602760: ✅ 완료

    - SRR28602761: ✅ 완료

    - SRR28602767: ✅ 완료

각 디렉토리 별로 아래 단계가 완료된 상태입니다.

- fastqc (품질 체크)

- trim_galore (트리밍)

- STAR (정렬 및 BAM 파일 생성)

- featureCounts (유전자 발현량 정량화, counts.txt 생성)

In [7]:
import pandas as pd
import glob
import os

tumor_paths = glob.glob('./Tumor/*/counts.txt')
normal_paths = glob.glob('./Normal/*/counts.txt')

usecols = ['Geneid', 'SRR']

counts_tumor = []
for path in tumor_paths:
    sample_name = os.path.basename(os.path.dirname(path))
    df = pd.read_csv(path, sep='\t', comment='#', usecols=[0, 6], names=['Geneid', sample_name], header=0)
    counts_tumor.append(df.set_index('Geneid'))

counts_tumor_df = pd.concat(counts_tumor, axis=1)

counts_normal = []
for path in normal_paths:
    sample_name = os.path.basename(os.path.dirname(path))
    df = pd.read_csv(path, sep='\t', comment='#', usecols=[0, 6], names=['Geneid', sample_name], header=0)
    counts_normal.append(df.set_index('Geneid'))

counts_normal_df = pd.concat(counts_normal, axis=1)

combined_counts_df = pd.concat([counts_tumor_df, counts_normal_df], axis=1)
combined_counts_df.columns.name = 'Sample'

print(combined_counts_df.head())

Sample           SRR30510137  SRR30510151  SRR30510181  SRR28602760  \
Geneid                                                                
ENSG00000279928            6            0            2            2   
ENSG00000228037            0            0            7           15   
ENSG00000142611         2403          735          168         1128   
ENSG00000284616            0            0            0            0   
ENSG00000157911         1369          537          889          805   

Sample           SRR28602761  SRR28602767  
Geneid                                     
ENSG00000279928            2            6  
ENSG00000228037            8            6  
ENSG00000142611         1247          504  
ENSG00000284616            0            0  
ENSG00000157911         1177         1301  


In [8]:
combined_counts_df.columns = [
    'Tumor_SRR30510137',
    'Tumor_SRR30510151',
    'Tumor_SRR30510181',
    'Normal_SRR28602760',
    'Normal_SRR28602761',
    'Normal_SRR28602767'
]

In [9]:
# Geneid가 인덱스인 상태에서 바로 저장
combined_counts_df.to_csv("Result/combined_counts_df.csv", index=True)

In [10]:
combined_counts_df

,Tumor_SRR30510137,Tumor_SRR30510151,Tumor_SRR30510181,Normal_SRR28602760,Normal_SRR28602761,Normal_SRR28602767
Geneid,,,,,,
ENSG00000279928,6,0,2,2,2,6
ENSG00000228037,0,0,7,15,8,6
ENSG00000142611,2403,735,168,1128,1247,504
ENSG00000284616,0,0,0,0,0,0
ENSG00000157911,1369,537,889,805,1177,1301
...,...,...,...,...,...,...
ENSG00000271254,1719,790,351,271,619,1450
ENSG00000275987,0,0,0,2,4,0
ENSG00000268674,0,0,0,0,0,0


### combined_counts_df 데이터에서 중복 Geneid 제거하기

In [11]:
combined_counts_df = pd.read_csv("Result/combined_counts_df.csv")

combined_counts_df

,Geneid,Tumor_SRR30510137,Tumor_SRR30510151,Tumor_SRR30510181,Normal_SRR28602760,Normal_SRR28602761,Normal_SRR28602767
0,ENSG00000279928,6,0,2,2,2,6
1,ENSG00000228037,0,0,7,15,8,6
2,ENSG00000142611,2403,735,168,1128,1247,504
3,ENSG00000284616,0,0,0,0,0,0
4,ENSG00000157911,1369,537,889,805,1177,1301
...,...,...,...,...,...,...,...
62749,ENSG00000271254,1719,790,351,271,619,1450
62750,ENSG00000275987,0,0,0,2,4,0
62751,ENSG00000268674,0,0,0,0,0,0
62752,ENSG00000277475,4,0,0,1,0,8


### 정규화(Normalization)

- RNA-Seq 데이터 분석의 필수 단계입니다. 
- 다양한 방법 중 CPM(Counts Per Million)과 log 변환을 추천합니다.

In [13]:
import numpy as np
import pandas as pd

combined_counts_df = pd.read_csv('Result/combined_counts_df.csv', index_col=0)

def cpm(counts):
    return (counts / counts.sum()) * 1e6

normalized_df = combined_counts_df.apply(cpm, axis=0)
normalized_log_df = np.log2(normalized_df + 1)

normalized_log_df.to_csv('Result/normalized_counts.csv', index=False)

print(normalized_log_df.head())

                 Tumor_SRR30510137  Tumor_SRR30510151  Tumor_SRR30510181  \
Geneid                                                                     
ENSG00000279928           0.114254           0.000000           0.083805   
ENSG00000228037           0.000000           0.000000           0.274213   
ENSG00000142611           5.087772           3.979975           2.590725   
ENSG00000284616           0.000000           0.000000           0.000000   
ENSG00000157911           4.307749           3.560476           4.785833   

                 Normal_SRR28602760  Normal_SRR28602761  Normal_SRR28602767  
Geneid                                                                       
ENSG00000279928            0.038284            0.031208            0.109349  
ENSG00000228037            0.265062            0.120976            0.109349  
ENSG00000142611            4.014973            3.871297            2.928702  
ENSG00000284616            0.000000            0.000000            0.000000  

### PCA(Principal Component Analysis)

정규화 후 PCA를 통해 샘플 간 유사성을 탐색하고, 각 군(Tumor vs Normal)의 명확한 분리를 확인할 수 있음

In [14]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('normalized_counts.csv')

pca = PCA(n_components=2)
pca_result = pca.fit_transform(data.T)

pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
pca_df['Sample'] = ['Tumor', 'Tumor', 'Tumor', 'Normal', 'Normal', 'Normal']

plt.figure(figsize=(8,6))
sns.scatterplot(data=pca_df, x='PC1', y='PC2', hue='Sample', s=100)

plt.title('PCA of Glioblastoma RNA-seq Samples')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Sample type')
plt.grid(True)

plt.savefig('PCA_plot.png')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'normalized_counts.csv'

### DESeq2 (R)

DESeq2는 RNA 시퀀싱 데이터를 분석할 때 사용하는 대표적인 R 패키지입니다. 

특히, **2개 이상의 그룹 간 유전자 발현 수준의 차이를 통게적으로 평가할 때 사용함.**

예를 들어, 질병 조직과 정상 조직 사이에서 어떤 유전자들이 통계적으로 의미 있게 차이가 나는지를 파악할 때 사용합니다.

이런 유전자들을 **차등 발현 유전자(Differentially Expressed Genes, DEGs)** 라고 부릅니다.

---

#### DESeq2를 사용하는 시점과 목적 

- RNA-seq 데이터를 분석하여 질병과 정상 조직 간의 유전자 차이 파악이 필요할 때 사용합니다. 

- 약물 처리군과 대조군 간의 유전자 발현 변화를 평가할 때 사용합니다. 

- 돌연변이 유무, 유전자 Knock-down, 유전자 Over-expression 등 실험 조건 간 유전자 발현 수준 비교 시 사용합니다. 

In [15]:
import pandas as pd

res_df = pd.read_csv('Result/DESeq2_results.csv', index_col=0)
res_df.head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
ENSG00000279928,2.758013,-0.405592,1.749998,-0.231767,0.816719,0.894558
ENSG00000228037,6.388304,-1.640095,1.805583,-0.908346,0.363695,0.533550
ENSG00000142611,921.031206,-0.018921,0.863661,-0.021908,0.982522,0.990358
ENSG00000284616,0.000000,NaN,NaN,NaN,NaN,NaN
ENSG00000157911,981.995279,-0.165080,0.582484,-0.283407,0.776864,0.869298


### R에서 DESeq2로 분석한 결과를 파이썬으로 불러오기 

- 유의한 DEGs를 필터링 

- 필터링을 통해 의미있는 유전자 집합을 추출하고, GO enrichment 또는 Pathway 분석 등의 추가 분석으로 나아갈 수 있습니다.

In [16]:
# 유의한 유전자만 필터링 (padj < 0.05 & 절대값 log2FoldChange > 1)
sig_genes = res_df[(res_df['padj'] < 0.05) & (abs(res_df['log2FoldChange']) > 1)]

# 종양에서 높게 발현된 유전자
upregulated = sig_genes[sig_genes['log2FoldChange'] > 0]

# 종양에서 낮게 발현된 유전자
downregulated = sig_genes[sig_genes['log2FoldChange'] < 0]

print("유의한 차등 발현 유전자 수:", len(sig_genes))
print("종양에서 증가된 유전자 수:", len(upregulated))
print("종양에서 감소된 유전자 수:", len(downregulated))

유의한 차등 발현 유전자 수: 8231
종양에서 증가된 유전자 수: 3287
종양에서 감소된 유전자 수: 4944


### 기능적 해석 (Functional Enrichment Analysis)

- 유의한 유전자들이 어떤 생물학적 과정(Gene Ontology, GO) 또는 대사 경로(Pathway)와 연관되어 있는지를 파악함.

#### GProfiler  

- "유전자 목록을 넣으면, 이 유전자들이 공통적으로 어떤 기능에 관여하는지"를 분석하는 프로그램입니다.

- 현재 Glioblastoma RNA-seq 분석에서 얻은 유의한 유전자 목록을 입력하고, 이 유전자들이 공통적으로 어떤 기능에 연관되어 있는지를 분석하려는 단계입니다.

- `sources` 옵션에는 분석 방법이나 알고리즘을 넣는 것이 아니라, 어떤 생물학적 데이터베이스를 이용해서 유전자 기능을 해석할지 지정하는 것입니다. 

- `query`는 유전자 목록을 넣고 `sources`는 분석에 사용할 데이터베이스를 넣음.

In [17]:
from gprofiler import GProfiler

significant_genes = sig_genes.index.tolist()

gp = GProfiler(return_dataframe=True)
enrichment_results = gp.profile(
    organism='hsapiens',
    query=significant_genes,
    sources=['GO:BP', 'KEGG', 'REAC']
)

print(enrichment_results.head())
enrichment_results.to_csv("enrichment_results.csv")

  source      native                              name       p_value  \
0  GO:BP  GO:0065007             biological regulation  1.059754e-50   
1  GO:BP  GO:0050789  regulation of biological process  7.477963e-47   
2  GO:BP  GO:0032502             developmental process  2.207014e-46   
3  GO:BP  GO:0048856  anatomical structure development  3.787809e-46   
4  GO:BP  GO:0050794    regulation of cellular process  1.424319e-43   

   significant                                        description  term_size  \
0         True  "Any process that modulates a measurable attri...      12671   
1         True  "Any process that modulates the frequency, rat...      12278   
2         True  "A biological process whose specific outcome i...       6478   
3         True  "The biological process whose specific outcome...       5924   
4         True  "Any process that modulates the frequency, rat...      11876   

   query_size  intersection_size  effective_domain_size  precision    recall  \
0     